In [2]:
import torch

import sys
sys.path.append('../')

from src.models import model_factory
from src.dataloaders import dataloader_factory
from src.datasets import dataset_factory
from src.trainers import trainer_factory
from src.utils.utils import *
from src.utils.options import parser

In [3]:
args = parser.parse_args([])

args.data_path = '../data/ml-1m' # "../data/Beauty"
args.num_epochs = 100
args.trm_max_len = 50

ckpt_root = setup_train(args)

Folder created: /home/zhankui/1_engineering/Qualcomm-FastRec/demo/experiments/test_2022-01-04_26
{'adam_epsilon': 1e-06,
 'best_metric': 'NDCG@10',
 'data_path': '../data/ml-1m',
 'dataloader_code': 'sasrec',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'item',
 'dataset_split_seed': 98765,
 'device': 'cpu',
 'device_idx': '0',
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'global_epochs': 1000,
 'local_epochs': 10,
 'lr': 0.001,
 'metric_ks': [5,
               10,
               20],
 'model_code': 'sasrec',
 'model_init_seed': 0,
 'num_epochs': 100,
 'num_gpu': 1,
 'optimizer': 'Adam',
 'split': 'leave_one_out',
 'subset_size': 1000,
 'test_batch_size': 64,
 'test_negative_sample_size': 100,
 'test_negative_sampler_code': 'random',
 'test_negative_sampling_seed': 98765,
 'train_batch_size': 64,
 'train_negative_sample_size': 100,
 'train_negative_sampler_code': 'random',
 'train_negative_sampling_seed': 0,
 'trainer_code': 'sasrec_sample',
 'trm_att_dropo

## Build Dataset

In [4]:
dataset = dataset_factory(args)
train_loader, val_loader, test_loader, dataset = dataloader_factory(args, dataset)

In [5]:
for batch in test_loader:
    break
users, seqs, candidates, labels, length = batch

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

## Evaluation Prep

In [6]:
model = model_factory(args)
trainer = trainer_factory(args, model, train_loader, val_loader, test_loader, ckpt_root, dataset.data)

from tqdm import tqdm
from src.utils.utils import AverageMeterSet

def evaluate(session, test_loader, metric_ks, ranker):
    average_meter_set = AverageMeterSet()

    with torch.no_grad():
        tqdm_dataloader = tqdm(test_loader)
        for batch_idx, batch in enumerate(tqdm_dataloader):
            users, seqs, candidates, labels, length = batch
            if users.size(0) != args.test_batch_size:
                continue

            ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
            scores = torch.Tensor(session.run(None, ort_inputs)[0])

            res = ranker(scores)
            metrics = {}
            for i, k in enumerate(metric_ks):
                metrics["NDCG@%d" % k] = res[2*i]
                metrics["Recall@%d" % k] = res[2*i+1]
            metrics["MRR"] = res[-3]
            metrics["AUC"] = res[-2]

            for k, v in metrics.items():
                average_meter_set.update(k, v)
            description_metrics = ['NDCG@%d' % k for k in metric_ks[:3]] +\
                                    ['Recall@%d' % k for k in metric_ks[:3]] + ['MRR'] + ['AUC'] + ['loss']
            description = 'FINAL TEST: ' + ', '.join(s + ' {:.5f}' for s in description_metrics)
            description = description.replace('NDCG', 'N').replace('Recall', 'R').replace('MRR', 'M').replace('Jaccard', 'J')
            description = description.format(*(average_meter_set[k].avg for k in description_metrics))
            tqdm_dataloader.set_description(description)

        average_metrics = average_meter_set.averages()

        return average_metrics

## Convert To FP16 Model

In [14]:
def convert_fp16():

    import onnx 
    import onnxruntime
    import onnxmltools
    from onnxmltools.utils.float16_converter import convert_float_to_float16

    onnx_model = onnx.load("model.onnx")
    onnx.checker.check_model(onnx_model)

    ort_session = onnxruntime.InferenceSession("model.onnx")

    # compute ONNX Runtime output prediction
    ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
    ort_outs = ort_session.run(None, ort_inputs)

    onnx_model = convert_float_to_float16(onnx_model)
    onnxmltools.utils.save_model(onnx_model, "model_fp16.onnx")

convert_fp16()

In [7]:
import onnxruntime 

ort_session_fp16 = onnxruntime.InferenceSession("model_fp16.onnx")
ort_session = onnxruntime.InferenceSession("model.onnx")

/home/zhankui/anaconda3/envs/bert4rec/lib/python3.6/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:353: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  "based on the build flags) when instantiating InferenceSession."


In [8]:

ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
ort_outs_fp16 = ort_session_fp16.run(None, ort_inputs)
ort_outs = ort_session.run(None, ort_inputs)

In [9]:
res_fp16 = evaluate(ort_session_fp16, test_loader, args.metric_ks, trainer.ranker)
res = evaluate(ort_session, test_loader, args.metric_ks, trainer.ranker)

FINAL TEST: N@5 0.36613, N@10 0.40811, N@20 0.43920, R@5 0.49102, R@10 0.62051, R@20 0.74318, M 0.35758, AUC 0.85303, loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 353.34it/s]
FINAL TEST: N@5 0.36651, N@10 0.40864, N@20 0.43956, R@5 0.49152, R@10 0.62151, R@20 0.74352, M 0.35790, AUC 0.85300, loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 374.43it/s]


In [10]:
print(f"AUC Change: \t {res_fp16['AUC'] / res['AUC']}")

AUC Change: 	 1.000040911340166


## Convert To Mixed-Precision Model

In [11]:
def convert_mixed():
    import onnx 
    from onnxruntime.quantization import quantize_dynamic, QuantType

    quantize_dynamic("model_fp16.onnx", "model_mixed.onnx", weight_type=QuantType.QInt8)

convert_mixed()

In [ ]:
import onnxruntime

ort_session_mixed = onnxruntime.InferenceSession("model_mixed.onnx")
ort_session = onnxruntime.InferenceSession("model.onnx")

ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
ort_outs_mixed = ort_session_mixed.run(None, ort_inputs)
print(ort_outs_mixed)

In [8]:
res_mixed = evaluate(ort_session_mixed, test_loader, args.metric_ks, trainer.ranker)
res = evaluate(ort_session, test_loader, args.metric_ks, trainer.ranker)

print(f"AUC Change: \t {res_mixed['AUC'] / res['AUC']}")

FINAL TEST: N@5 0.02813, N@10 0.15688, N@20 0.24542, R@5 0.04375, R@10 0.43542, R@20 0.78542, M 0.10499, AUC 0.77461, loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 143.71it/s]
FINAL TEST: N@5 0.03037, N@10 0.16213, N@20 0.24740, R@5 0.04792, R@10 0.44688, R@20 0.78438, M 0.10727, AUC 0.77543, loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 435.79it/s]

AUC Change: 	 0.9989521714670064
